### Download input files and required modules

In [ ]:
# retrieve the sp code
!rm -rf sp && mkdir sp
!wget "https://github.com/UCB-CE170a/Fall2020/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/interface.py" -O sp/interface.py

# retrieve the spatial queue model
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/queue_model.py" -O queue_model.py

# retrieve the road network
!rm -rf traffic_inputs && mkdir traffic_inputs
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_edges.csv" -O traffic_inputs/berkeley_edges.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_nodes.csv" -O traffic_inputs/berkeley_nodes.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/od_20pn.csv" -O traffic_inputs/od_20pn.csv

# create output directory
!rm -rf traffic_outputs
!mkdir -p traffic_outputs
!mkdir -p traffic_outputs/t_stats
!mkdir -p traffic_outputs/node_stats
!mkdir -p traffic_outputs/link_stats

### Import modules

In [ ]:
import time 
import random
import logging 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
random.seed(0)
np.random.seed(0)

### spatial queue model
from queue_model import Node, Link, Agent, Simulation

### Run the simulation
Note: if you change anything below, you need to re-run all cells start from here

In [ ]:
#################### Modify here #################### 
### your name in the string, avoid space
yourname = 'tester'
### scenario you are running, choose between "base" and "contraflow"
scenario = 'base'
### simulation duration, specify at which time step to stop the simulation. It needs to be long enough for all evacuees to arrive at the safe destination.
t_end = 3601
####################  End here  #################### 
if scenario =='contraflow': print('please upload your `contraflow_edges_{}.csv` under the `traffic_inputs` folder'.format(yourname))

The code block below reads in the input files. If you are running the `contraflow` scenario, the `contraflow_edges_[yourname].csv` file storing the contraflow information will be read in as well. The road network graph used for traffic simulation is then generated based on both the base network file and the new contraflow information.

In [ ]:
nodes_df = pd.read_csv('traffic_inputs/berkeley_nodes.csv')
links_df = pd.read_csv('traffic_inputs/berkeley_edges.csv')
od_df = pd.read_csv('traffic_inputs/od_20pn.csv')

if scenario == 'base':
    pass
elif scenario == 'contraflow':
    contraflow_links_df = pd.read_csv('traffic_inputs/contraflow_edges_{}.csv'.format(yourname))
    print('Your choice of contraflow locations are \n')
    display(contraflow_links_df.head())
    links_df = links_df.merge(contraflow_links_df[['link_id', 'new_lanes']], how='left', on='link_id')
    links_df['lanes'] = np.where(np.isnan(links_df['new_lanes']), links_df['lanes'], links_df['new_lanes'])
else: 
    print('invalid scenario, run base scenario')
scenario_name = '{}_{}'.format(scenario, yourname)

simulation = Simulation()
simulation.create_network(nodes_df, links_df)
simulation.create_demand(od_df)

In [ ]:
### calculating shortest path
cannot_find_path = []
for agent_id, agent in simulation.all_agents.items():
    routing_status = agent.get_path( g=simulation.g )
    if routing_status == 'no_path_found':
        cannot_find_path.append(agent_id)

for agent_id in cannot_find_path:
    del simulation.all_agents[agent_id]
print('# o-d pairs whose paths cannot be found: {}'.format(len(cannot_find_path)))
print('# o-d pairs/trips {}'.format(len(simulation.all_agents)))

In [ ]:
### run spatial queue simulation
reroute_frequency = 3600 ### time unit is second

### a file that stores the numbers of arrived evacuees at different time steps
with open('traffic_outputs/t_stats/t_stats_{}.csv'.format(scenario_name), 'w') as t_stats_outfile:
    t_stats_outfile.write("t,arr"+"\n")

for t in range(t_end):
  
    ### load agents
    for agent_id, agent in simulation.all_agents.items(): 
        agent.load_trips(t)
        ### reroute
        if (t>0) and (t%reroute_frequency == 0):
            routing_status = agent.get_path( g=simulation.g )

    ### run link model
    for link_id, link in simulation.all_links.items():
        link.run_link_model(t)

    ### run node model
    node_ids_to_run = set([link.end_nid for link in simulation.all_links.values() if len(link.queue_veh)>0])
    for node_id in node_ids_to_run:
        node = simulation.all_nodes[node_id] 
        node.run_node_model(t)
    
    ### output time-step results
    if t%100 == 0:
        arrival_cnts = np.sum([1 for a in simulation.all_agents.values() if a.status=='arr'])
        print('At {} seconds, {} evacuees successfully reached the destination'.format(t, arrival_cnts))
        if arrival_cnts == len(simulation.all_agents):
            print("all agents arrive at destinations at time {} seconds.".format(t))
            break
        with open('traffic_outputs/t_stats/t_stats_{}.csv'.format(scenario_name), 'a') as t_stats_outfile:
            t_stats_outfile.write("{},{}".format(t, arrival_cnts) + "\n")
    
    ### output queue length for each link
    if t%100 == 0:
        link_output = pd.DataFrame([(link.lid, len(link.queue_veh), len(link.run_veh), link.geometry) for link in simulation.all_links.values() if link.ltype[0:2]!='vl'], columns=['link_id', 'q', 'r', 'geometry'])
        link_output[(link_output['q']>0) | (link_output['r']>0)].reset_index(drop=True).to_csv('traffic_outputs/link_stats/link_stats_{}_t{}.csv'.format(scenario_name, t), index=False)
        node_predepart = pd.DataFrame([(agent.cle, 1) for agent in simulation.all_agents.values() if (agent.status in [None, 'loaded'])], columns=['node_id', 'predepart_cnt']).groupby('node_id').agg({'predepart_cnt': np.sum}).reset_index()
        if node_predepart.shape[0]>0:
            node_predepart.merge(nodes_df[['node_id', 'lat', 'lon']], how='left', on='node_id').to_csv('traffic_outputs/node_stats/node_stats_{}_t{}.csv'.format(scenario_name, t), index=False)

### Visualize the results
There are at least two types of visualizations that you can make based on the output files from running the simulation code above:

1. Arrival curve. This is a line plot showing the number of arrived vehicles at different time steps. The code block below shows an example of how to make the arrival plot.

2. Queueing status and departure status map. Suppose we want to visualize the queueing status 20 minutes after the onset of the evacuation under the base scenario in QGIS, we just need to load the file `link_stats_base_t1000.csv` to QGIS and choose a suitable categorized legend. Similarly, we can also load `node_stats_base_t1000.csv` to the same map to display the number of vehicles that are still at their origin node (cannot departure because the road in front of their origin is too congested).

These visualizations can help you to get an intuitive idea of whether your strategy is effective

In [ ]:
### plot the arrival curve
fig, ax = plt.subplots(figsize=(10,5))

### base scenario arrival
t_stats_base_df = pd.read_csv('traffic_outputs/t_stats/t_stats_{}.csv'.format('base_tester'))
ax.plot(t_stats_base_df['t'], t_stats_base_df['arr'], label='base')

### contraflow scenario arrival
t_stats_contraflow_df = pd.read_csv('traffic_outputs/t_stats/t_stats_{}.csv'.format('contraflow_tester'))
ax.plot(t_stats_contraflow_df['t'], t_stats_contraflow_df['arr'], label='contraflow')

# t_stats_contraflow_df = pd.read_csv('traffic_outputs/t_stats/t_stats_{}.csv'.format('contraflow_tester_2'))
# ax.plot(t_stats_contraflow_df['t'], t_stats_contraflow_df['arr'], label='contraflow_2')

ax.set_xlabel('Time since the start of the evacuation')
ax.set_ylabel('# arrived vehicles')
ax.set_xlim([0, 3600])
plt.legend()
plt.show()

In [ ]:
### run this command to put all the outputs in a compressed file
### you can then download the file to your local computer for visualization
!tar czvf traffic_outputs.tar.gz traffic_outputs